In [ ]:
!pip uninstall torch torchvision torchaudio

In [ ]:
!pip cache purge

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

In [ ]:
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

In [6]:
import firebase_admin
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from firebase_admin import credentials, firestore
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import math
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Firebase Initialization
cred = credentials.Certificate("adminkey.json")
#firebase_admin.initialize_app(cred)
db = firestore.client()

# Constants
INCLUDE_ONLY = ['44444']
ACTIVITIES = ['sit', 'walk', 'upstairs']
SAMPLING_RATE = 100  # Hz
OVERLAP = 0.5
CHUNK_SIZE = 1  # seconds
BATCH_SIZE = 64
EPOCHS = 30
LEARNING_RATE = 0.0005

scaler = StandardScaler()

NameError: name '_C' is not defined

In [ ]:
def fetch_data_by_treatment(treatment_filter):
    data_raw, docs = [], []
    data_ref = db.collection("data")
    treatment_doc = data_ref.document(treatment_filter)

    for patient_id in INCLUDE_ONLY:
        subject_ref = treatment_doc.collection(patient_id)
        for activity in ACTIVITIES:
            activity_ref = subject_ref.document(activity)
            phone_location_col = activity_ref.collections()
            for phone_location in phone_location_col:
                for recording in phone_location.stream():
                    record = recording.to_dict()
                    if 'acceleration' not in record:
                        continue
                    df = pd.DataFrame(record['acceleration'])
                    df = df.iloc[25:-25]  # remove first and last 2.5s if needed
                    combined = pd.DataFrame({
                        'ax': df['accelerometer'].apply(lambda x: x['x']),
                        'ay': df['accelerometer'].apply(lambda x: x['y']),
                        'az': df['accelerometer'].apply(lambda x: x['z']),
                        'gx': df['gyroscope'].apply(lambda x: x['x']),
                        'gy': df['gyroscope'].apply(lambda x: x['y']),
                        'gz': df['gyroscope'].apply(lambda x: x['z']),
                    })
                    combined_scaled = pd.DataFrame(scaler.fit_transform(combined), columns=combined.columns)
                    data_raw.append(combined_scaled)
                    docs.append({'activity': activity})
    return data_raw, docs

def chunk_data(data_raw, docs, chunk_size_sec, sampling_rate, overlap=OVERLAP):
    data, labels = [], []
    chunk_samples = int(chunk_size_sec * sampling_rate)
    step = int(chunk_samples * (1 - overlap))
    for i, df in enumerate(data_raw):
        for start in range(0, len(df) - chunk_samples + 1, step):
            chunk = df.iloc[start:start + chunk_samples]
            if len(chunk) == chunk_samples:
                data.append(chunk.values)
                labels.append(ACTIVITIES.index(docs[i]['activity']))
    return np.array(data), np.array(labels)

def extract_features(chunk):
    features = []
    for i in range(chunk.shape[1]):
        col = chunk[:, i]
        features.extend([col.mean(), np.median(col), col.std(), col.var(), col.min(), col.max()])
    return features

In [ ]:
def prepare_knn_data(data_raw, docs, chunk_size_sec, sampling_rate, overlap=OVERLAP):
    X, y = [], []
    chunk_samples = int(chunk_size_sec * sampling_rate)
    step = int(chunk_samples * (1 - overlap))
    for i, df in enumerate(data_raw):
        for start in range(0, len(df) - chunk_samples + 1, step):
            chunk = df.iloc[start:start + chunk_samples].values
            if len(chunk) == chunk_samples:
                X.append(extract_features(chunk))
                y.append(ACTIVITIES.index(docs[i]['activity']))
    return np.array(X), np.array(y)

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(6, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * (CHUNK_SIZE * SAMPLING_RATE // 4), 128)
        self.fc2 = nn.Linear(128, len(ACTIVITIES))
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # batch x channels x seq_len
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

In [ ]:
class CNNLSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(6, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(2)
        self.lstm = nn.LSTM(32, 64, batch_first=True)
        self.fc = nn.Linear(64, len(ACTIVITIES))
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = x.permute(0, 2, 1)
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = self.dropout(x)
        x = self.fc(x)
        return x

In [7]:
def train_dl_model(model_class, X_train, y_train, X_test, y_test):
    model = model_class().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    train_ds = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    model.train()
    for epoch in range(EPOCHS):
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {loss.item():.4f}")
    # Evaluation
    model.eval()
    with torch.no_grad():
        preds = model(torch.tensor(X_test, dtype=torch.float32).to(device)).argmax(dim=1).cpu().numpy()
    print(classification_report(y_test, preds, target_names=ACTIVITIES))
    cm = confusion_matrix(y_test, preds)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=ACTIVITIES, yticklabels=ACTIVITIES)
    plt.title(f"{model_class.__name__} Confusion Matrix")
    plt.show()

In [8]:
if __name__ == "__main__":
    # Fetch and chunk data
    before_raw, before_docs = fetch_data_by_treatment('BeforeTreatment')
    after_raw, after_docs = fetch_data_by_treatment('AfterTreatment')
    X_train_cnn, y_train = chunk_data(before_raw, before_docs, CHUNK_SIZE, SAMPLING_RATE)
    X_test_cnn, y_test = chunk_data(after_raw, after_docs, CHUNK_SIZE, SAMPLING_RATE)

    print("\n=== Running CNN Model ===")
    train_dl_model(SimpleCNN, X_train_cnn, y_train, X_test_cnn, y_test)

    print("\n=== Running CNN-LSTM Model ===")
    train_dl_model(CNNLSTM, X_train_cnn, y_train, X_test_cnn, y_test)

    # KNN
    print("\n=== Running KNN Model ===")
    X_train_knn, y_train_knn = prepare_knn_data(before_raw, before_docs, CHUNK_SIZE, SAMPLING_RATE)
    X_test_knn, y_test_knn = prepare_knn_data(after_raw, after_docs, CHUNK_SIZE, SAMPLING_RATE)
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(X_train_knn, y_train_knn)
    preds_knn = knn.predict(X_test_knn)
    print(classification_report(y_test_knn, preds_knn, target_names=ACTIVITIES))
    cm_knn = confusion_matrix(y_test_knn, preds_knn)
    sns.heatmap(cm_knn, annot=True, fmt='d', cmap='Blues', xticklabels=ACTIVITIES, yticklabels=ACTIVITIES)
    plt.title("KNN Confusion Matrix")
    plt.show()

NameError: name 'fetch_data_by_treatment' is not defined